# Utils


In [ ]:
# | default_exp indexing.utils

In [ ]:
# | export

from dreamai.imports import *
from langchain_ray.imports import *
from langchain_ray.utils import *
from langchain_ray.chains import *

In [ ]:
# | export


def index_names(index_folder, index_name):
    return [
        f.stem for f in Path(index_folder).glob("*.faiss") if f.stem.startswith(index_name)
    ]


def docs_ems(docs, ems_model):
    return ems_model.encode([doc.page_content for doc in docs]).tolist()


def df_docs_ems(df, ems_model, ems_folder, docs_col="doc"):
    os.makedirs(ems_folder, exist_ok=True)
    ems = docs_ems(df[docs_col], ems_model)
    for doc, em in zip(df[docs_col], ems):
        source = Path(doc.metadata["source"])
        ems_path = (Path(ems_folder) / source.stem).with_suffix(".json")
        if ems_path.exists():
            ems_path = find_alternate_path(ems_path, first_idx=1, verbose=False)
        with open(ems_path, "w") as f:
            ems_dict = {"embedding": em, "metadata": doc.metadata}
            json.dump(ems_dict, f)
    return df


def docs_to_faiss(
    docs,
    ems_model,
    index_folder="/media/hamza/data2/faiss_data/saved_indexes/",
    index_name="index",
):
    db = FAISS.from_documents(docs, ems_model)
    index_path = (Path(index_folder) / index_name).with_suffix(".faiss")
    if index_path.exists():
        index_path = find_alternate_path(index_path, first_idx=1, verbose=False)
    index_name = index_path.stem
    db.save_local(index_folder, index_name)
    return docs


def search_faiss(index_folder, index_names, query, ems_model, filter=None, k=2):
    # print(f"\n\nSearching {index_names} in {index_folder} for {query}!!!\n\n")
    if path_or_str(index_folder):
        index_folder = [index_folder]
    if path_or_str(index_names):
        index_names = [index_names]
    if path_or_str(query):
        query = [query]
    if is_iter(k):
        k = int(k[0])
    q_sims = []
    for index_folder, index_name, query in zip(index_folder, index_names, query):
        # print(f"\n\nSearching {index_name} in {index_folder} for {query}\n\n")
        db = FAISS.load_local(index_folder, embeddings=ems_model, index_name=index_name)
        q_sims.append(db.similarity_search_with_score(query, filter=filter, k=k))
    # print(f"\n\nQSIMS: {len([q_sims])}, INDEXES: {len(index_names)}\n\n")
    return [[q_sims]]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()